# Algoritmo 3 - Associação de Regras

Vamos usar o Apriori para encontrar associações entre os desportos praticados pelos atletas olímpicos.

In [13]:
print("Hello World!")

%pip install --upgrade pip
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install mlxtend


Hello World!


/opt/homebrew/Cellar/python@3.14/3.14.0_1/Frameworks/Python.framework/Versions/3.14/lib/python3.14/pty.py:66: DeprecationWarning: This process (pid=31736) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


/opt/homebrew/Cellar/python@3.14/3.14.0_1/Frameworks/Python.framework/Versions/3.14/lib/python3.14/pty.py:66: DeprecationWarning: This process (pid=31736) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.


/opt/homebrew/Cellar/python@3.14/3.14.0_1/Frameworks/Python.framework/Versions/3.14/lib/python3.14/pty.py:66: DeprecationWarning: This process (pid=31736) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


/opt/homebrew/Cellar/python@3.14/3.14.0_1/Frameworks/Python.framework/Versions/3.14/lib/python3.14/pty.py:66: DeprecationWarning: This process (pid=31736) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


# Importar Ficheiro CSV

In [14]:
import pandas as pd

winterData = pd.read_csv("Data/Olympics/winter.csv")
print(winterData.head())

# Add new column
winterData["Season"] = "Winter"
print(winterData.head())



summerData = pd.read_csv("Data/Olympics/summer.csv")
print(summerData.head())

summerData["Season"] = "Summer"
print(summerData.head())




concatData = pd.concat([winterData, summerData], ignore_index=True)



print("---------------------------------------------")

concatData.rename(columns={"Country": "Country_Code"}, inplace=True)
print(concatData.tail())


dictionary = pd.read_csv("Data/Olympics/dictionary.csv")

olympics = concatData.merge(
    dictionary[["Code","Country", "Population", "GDP per Capita"]],
    left_on="Country_Code",   # HUN, AUT, GRE, ...
    right_on="Code",     # HUN, AUT, GRE, ...
    how="left"
)






   Year      City     Sport Discipline               Athlete Country Gender  \
0  1924  Chamonix  Biathlon   Biathlon           BERTHET, G.     FRA    Men   
1  1924  Chamonix  Biathlon   Biathlon        MANDRILLON, C.     FRA    Men   
2  1924  Chamonix  Biathlon   Biathlon   MANDRILLON, Maurice     FRA    Men   
3  1924  Chamonix  Biathlon   Biathlon       VANDELLE, André     FRA    Men   
4  1924  Chamonix  Biathlon   Biathlon  AUFDENBLATTEN, Adolf     SUI    Men   

             Event   Medal  
0  Military Patrol  Bronze  
1  Military Patrol  Bronze  
2  Military Patrol  Bronze  
3  Military Patrol  Bronze  
4  Military Patrol    Gold  
   Year      City     Sport Discipline               Athlete Country Gender  \
0  1924  Chamonix  Biathlon   Biathlon           BERTHET, G.     FRA    Men   
1  1924  Chamonix  Biathlon   Biathlon        MANDRILLON, C.     FRA    Men   
2  1924  Chamonix  Biathlon   Biathlon   MANDRILLON, Maurice     FRA    Men   
3  1924  Chamonix  Biathlon   Biath

In [15]:
from mlxtend.preprocessing import TransactionEncoder


olympics["SportMedal"] = olympics["Sport"] + "_" + olympics["Medal"]

transactions_df = (
    olympics
    .groupby(["Country", "Year", "Season"])["SportMedal"]
    .unique()
    .reset_index()
)


transactions_df["num_items"] = transactions_df["SportMedal"].apply(len)
transactions_df = transactions_df[transactions_df["num_items"] >= 1].reset_index(drop=True)

transactions = transactions_df["SportMedal"].tolist()


te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
basket = pd.DataFrame(te_ary, columns=te.columns_)


In [17]:
from mlxtend.frequent_patterns import apriori, association_rules


frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

rules["antecedent_len"] = rules["antecedents"].apply(len)
rules["consequent_len"] = rules["consequents"].apply(len)

simple_rules = rules[
    (rules["antecedent_len"] == 1) &    # antecedente só com 1 item
    (rules["consequent_len"] == 1) &    # consequente com 1 item (se quiseres simples)
    (rules["support"] >= 0.02) &        # suporte mínimo
    (rules["lift"] > 1.5)               # lift para garantir associação interessante
].sort_values("lift", ascending=False)



rules["antecedent_len"] = rules["antecedents"].apply(len)
rules["consequent_len"] = rules["consequents"].apply(len)

rules_sorted = rules.sort_values("lift", ascending=False)
rules_sorted.head(20)
print(simple_rules.head(20))


                antecedents             consequents  antecedent support  \
108       (Equestrian_Gold)     (Equestrian_Silver)            0.051173   
109     (Equestrian_Silver)       (Equestrian_Gold)            0.066809   
100    (Canoe / Kayak_Gold)  (Canoe / Kayak_Bronze)            0.049751   
118     (Gymnastics_Bronze)     (Gymnastics_Silver)            0.078891   
119     (Gymnastics_Silver)     (Gymnastics_Bronze)            0.068941   
110          (Fencing_Gold)        (Fencing_Bronze)            0.063966   
111        (Fencing_Bronze)          (Fencing_Gold)            0.074627   
114        (Fencing_Silver)          (Fencing_Gold)            0.076048   
115          (Fencing_Gold)        (Fencing_Silver)            0.063966   
117       (Gymnastics_Gold)     (Gymnastics_Bronze)            0.066809   
116     (Gymnastics_Bronze)       (Gymnastics_Gold)            0.078891   
123     (Gymnastics_Silver)       (Gymnastics_Gold)            0.068941   
124       (Gymnastics_Gol